In [1]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
ex = "006"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [3]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 10000,
    'early_stopping_rounds': 200,
    'verbose_eval': 200,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",
                "../output/fe/fe004.feather",
                "../output/fe/fe005.feather",
                "../output/fe/fe006.feather", ]

In [4]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-17 11:11:15,505 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [7]:
y

0       8.78
1       8.06
2       7.90
3       6.38
4       8.17
        ... 
4995    7.23
4996    6.54
4997    6.51
4998    5.92
4999    6.83
Name: Score, Length: 5000, dtype: float64

In [8]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.285897	valid_1's rmse: 0.392729
[400]	training's rmse: 0.214328	valid_1's rmse: 0.38461
[600]	training's rmse: 0.172015	valid_1's rmse: 0.384477
Early stopping, best iteration is:
[423]	training's rmse: 0.208559	valid_1's rmse: 0.383885


2021-08-17 11:11:20,701 - INFO - Fold0:CV=0.3838852155306234
2021-08-17 11:11:20,719 - INFO - [fold 0] done in 5 s


Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.284756	valid_1's rmse: 0.391963
[400]	training's rmse: 0.213721	valid_1's rmse: 0.38575
[600]	training's rmse: 0.171963	valid_1's rmse: 0.384776
[800]	training's rmse: 0.142206	valid_1's rmse: 0.385931
Early stopping, best iteration is:
[636]	training's rmse: 0.165865	valid_1's rmse: 0.384531


2021-08-17 11:11:27,241 - INFO - Fold1:CV=0.38453097338625936
2021-08-17 11:11:27,247 - INFO - [fold 1] done in 7 s


Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.283877	valid_1's rmse: 0.413653
[400]	training's rmse: 0.215144	valid_1's rmse: 0.40395
[600]	training's rmse: 0.173351	valid_1's rmse: 0.399898
[800]	training's rmse: 0.144191	valid_1's rmse: 0.399113
[1000]	training's rmse: 0.121436	valid_1's rmse: 0.397986
[1200]	training's rmse: 0.103812	valid_1's rmse: 0.397544
[1400]	training's rmse: 0.0900832	valid_1's rmse: 0.397437
Early stopping, best iteration is:
[1349]	training's rmse: 0.0932626	valid_1's rmse: 0.397221


2021-08-17 11:11:40,554 - INFO - Fold2:CV=0.3972211680722735
2021-08-17 11:11:40,565 - INFO - [fold 2] done in 13 s


Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.284556	valid_1's rmse: 0.402898
[400]	training's rmse: 0.215109	valid_1's rmse: 0.397936


2021-08-17 11:11:45,793 - INFO - Fold3:CV=0.3973584557188197
2021-08-17 11:11:45,804 - INFO - [fold 3] done in 5 s


Early stopping, best iteration is:
[379]	training's rmse: 0.220611	valid_1's rmse: 0.397358
Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.278559	valid_1's rmse: 0.422297
[400]	training's rmse: 0.209538	valid_1's rmse: 0.411156
[600]	training's rmse: 0.167654	valid_1's rmse: 0.410237
[800]	training's rmse: 0.138785	valid_1's rmse: 0.410893
Early stopping, best iteration is:
[612]	training's rmse: 0.165651	valid_1's rmse: 0.409967


2021-08-17 11:11:52,996 - INFO - Fold4:CV=0.40996662311556714
2021-08-17 11:11:53,006 - INFO - [fold 4] done in 7 s
2021-08-17 11:11:53,017 - INFO - CV=0.39471076979507547
2021-08-17 11:11:53,019 - INFO - [lightgbm] done in 37 s


In [9]:
y_test_sub =  np.mean(y_test,axis=0)
sub = pd.read_csv(SUB_PATH)

In [10]:
sub["Score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [11]:
sub

,MAL_ID,Score
0,5,8.232569
1,6,8.120243
2,7,7.397372
3,8,6.568827
4,15,7.368938
...,...,...
7416,47307,7.184139
7417,47398,7.326218
7418,47402,7.499601
7419,47616,6.961976
